##### CODE BELOW ARE COPIED FROM DAY1

In [1]:
# direct sum
function ⨁(𝑨::AbstractArray{𝕋, ℕ}, 𝑩::AbstractArray{𝕋, ℕ}, axes::Tuple) where {𝕋, ℕ}
    𝑪 = []

    s𝑨 = size(𝑨) # store size of 𝑨 and 𝑩 to calculate size of 𝑪
    s𝑩 = size(𝑩)
    s𝑪 = []

    if length(axes) == ℕ
        s𝑪 = [x+y for (x, y) in zip(s𝑨, s𝑩)]
        𝑪 = zeros(s𝑪...)
        𝑩start = CartesianIndex([x+1 for x in s𝑨]...)
        𝑩end = CartesianIndex(s𝑪...)
    else
        if sum([[x!=y for (x, y) in zip(s𝑨, s𝑩)][x] for x in axes]) != 0
            error("parameter wrong")
        end

        # if x in axes, this dim's data will not expand
        #for x in 1:N if x in axes push!(s𝑪, s𝑨[x]) else push!(s𝑪, s𝑨[x]+s𝑩[x]) end end
        [x in axes ? s𝑪 : s𝑨[x]+s𝑩[x] for x in 1:N] # NOTES:P2
        𝑪 = zeros(s𝑪...)

        # construct 𝑩start
        tmp = ones(Int64, N) # NOTES:P1
        for x in 1:N if !(x in axes) tmp[x]=1+s𝑨[x] end end
        𝑩start = CartesianIndex((tmp...,))
        𝑩end = CartesianIndex(s𝑪...)
    end

    𝑨start = CartesianIndices(𝑪)[1]
    𝑨end = CartesianIndex(s𝑨...)
    𝑪[𝑨start:𝑨end] = 𝑨
    𝑪[𝑩start:𝑩end] = 𝑩

    return 𝑪
end

function contract!(a::AbstractArray, b::AbstractArray, dima::Tuple, dimb::Tuple, debug=false)
    if [size(a)[x] for x in dima] != [size(b)[x] for x in dimb] error("size is wrong") end
    if length(dima) == 0
        a_size = [size(a)...]
        b_size = [size(b)...]
        ra = reshape(a, prod(a_size), 1)
        rb = reshape(b, 1, prod(b_size))
        return reshape(ra*rb, Tuple([a_size; b_size]))
    end

    a_left = filter(x -> !(x in dima), [x for x in 1:ndims(a)])
    a_right = collect(dima)
    a_perm = [a_left; a_right]
    a_reshape_size = [size(a)[x] for x in a_left]
    a_left_len = prod(a_reshape_size)
    a_right_len = prod([size(a)[x] for x in a_right])

    b_left = collect(dimb)
    b_right = filter(x -> !(x in dimb), [x for x in 1:ndims(b)])
    b_perm = [b_left; b_right]
    b_left_len = prod([size(b)[x] for x in b_left])
    b_reshape_size = [size(b)[x] for x in b_right]
    b_right_len = prod(b_reshape_size)

    am = permutedims(a, a_perm)
    bm = permutedims(b, b_perm)

    ra = reshape(a, (a_left_len, a_right_len))
    rb = reshape(b, (b_left_len, b_right_len))
    
    return reshape(ra*rb, Tuple([a_reshape_size; b_reshape_size]))
end

# The right version for kron
function ⨂(t1::AbstractArray, t2::AbstractArray)
    if ndims(t1) != ndims(t2)
        error("ndims t1 must be equaled to t2")
    end

    T = promote_type(eltype(t2), eltype(t1))
    ret = zeros(T, size(t1).*size(t2))
    for i in CartesianIndices(t1)
        c_start = CartesianIndex(.+(.*(.-(Tuple(i), 1), size(t2)), 1))
        c_end = CartesianIndex(Tuple(i).*size(t2))
        ret[c_start:c_end] = t1[i] * t2
    end
    return ret
end

⨂ (generic function with 1 method)

##### HOMEWORK
1. single-qubit operation
2. two-qubits operation
3. measurement

In [2]:
#=

Convention for MPS

     i₂
     |
     |
i₁---⊕---i₃

=#
struct MPS
    data::Vector{Array{Number, 3}}
end

#=

Convention for OnebodyGate -- Gate₁

     i₁
     |
     ⊕
     |
     i₂

=#
struct Gate₁
    op::Array{𝕋, 2} where {𝕋<:Complex{Float64}}
end

#=

Convention for TwobodyGate -- Gate₂

     i₁ i₂
     |  |
    ------
    ------
     |  |
     i₃ i₄

=#
struct Gate₂
    i::Array{Int64, 1}
    op::Array{𝕋, 4} where {𝕋<:Complex{Float64}}
end

In [10]:
function initmps(basis::AbstractArray)
    d = [x==0 ? reshape([1,0], 1, 2, 1) : reshape([0,1], 1, 2, 1) for x in basis]
    return MPS(d)
end

# 1:2:1 contract! with 2:2 -> 1:1:2, so we need to reshape to 1:2:1 and reset the tensor
function 𝑿(sys::MPS, i::Int64)
    sys.data[i] = permutedims(contract!(sys.data[i], [0 1;1 0], (2,), (2,)), [1,3,2])
end

function 𝒀(sys::MPS, i::Int64)
    sys.data[i] = permutedims(contract!(sys.data[i], [0 -im;im 0], (2,), (2,)), [1,3,2])
end

function 𝒁(sys::MPS, i::Int64)
    sys.data[i] = permutedims(contract!(sys.data[i], [1 0;0 -1], (2,), (2,)), [1,3,2])
end

function 𝑯(sys::MPS, i::Int64)
    sys.data[i] = permutedims(contract!(sys.data[i], 1/sqrt(2)*[1 1;1 -1], (2,), (2,)), [1,3,2])
end

import LinearAlgebra:svd, Diagonal

function tsvd(a::AbstractArray{𝕋, ℕ}, axes::Tuple) where {𝕋, ℕ}
    a = Number.(a)
    sizeA = size(a)
    laxes = filter(x -> !(x in axes), collect(1:ℕ)) # not in the axes, permute to left side
    raxes = [axes...] # in the axes, permute to right side
    
    lsize = [sizeA[x] for x in laxes] # store original size for reshaping after the svd is done
    rsize = [sizeA[x] for x in raxes]
    
    # permute A, reshape to matrix then do svd
    u, s, vt = svd(reshape(permutedims(a, [laxes;raxes]), prod(lsize), prod(rsize)))
    u = u * sqrt(Diagonal(s)) # s will be absorbed into u and vt
    vt = vt * sqrt(Diagonal(s))
    
    # reshape to original order
    dim = length(s)
    return (reshape(u, lsize..., dim), reshape(vt, dim, rsize...))
end

# only can swap closed tensor
function swap_closed(sys::MPS, i₁::Int64, i₂::Int64)
    i₁, i₂ = min(i₁, i₂), max(i₁, i₂)
    if i₂ - i₁ != 1 error("parameter wrong") end
    contract!(sys.data[i₁], sys.data[i₂], (3,), (1,))
    res2g = contract!(sys.data[i₁], sys.data[i₂], (3,), (1,))
    swapg = reshape([1 0 0 0;0 0 1 0;0 1 0 0;0 0 0 1], 2, 2, 2, 2)
    tmp = contract!(res2g, swapg, (2, 3), (3, 4))
    u, vt = tsvd(reshape(tmp, 1, 2, 2, 1), (2, 4)) # ERROR!!!!
    # u is okay with convention, vt need to permute
    vt = permutedims(vt, (1,3,2))
    sys.data[i₁] = u
    sys.data[i₂] = vt
    return (u, vt)
end

# get one controlled single-qubit gate
function controlled(gate::AbstractArray)
    if size(gate) !== (2,2) error("parameter wrong") end
    # generate |0><0|⨂Identity
    a = contract!([1 0;0 0], [1 0;0 1], (), ())
    # generate |1><1|⨂OnebodyGate
    b = contract!([0 0;0 1], gate, (), ())
    return permutedims(a+b, [1,3,2,4])
end

function CNOT(sys::MPS, con::Int64, op::Int64)
    g₂ = controlled([0 1;1 0])
    sites = contract!(sys.data[con], sys.data[op], (3,), (1,))
    sites = contract!(sites, g₂, (2,3), (3,4))
    u, vt = tsvd(sites, (4, 2))
    sys.data[con] = u
    sys.data[op] = vt
    return (u, vt)
end

# not finished
function projection(sys::MPS, proj::MPS)
    length(sys.data) != length(proj.data) && error("system mps and projection mps's dims must be equal")
    
end

# not finished
function measure(sys::MPS, i::Int64)
    permutedims(contract!(sys.data[i], [1 0;0 0], (2,), (2,)), [1,3,2])
end

measure (generic function with 1 method)

In [18]:
function testEPR(basis::AbstractArray)
    proj = initmps(basis)
    t = contract!(mps.data[2], proj.data[2], (2,), (2,))
    t = contract!(t, mps.data[1], (1,), (3,))
    t = contract!(t, proj.data[1], (2,5), (1,2))
    return reshape(t, 1)[1]
end

mps = initmps([0,0])
𝑯(mps, 1)
CNOT(mps, 1, 2)
println(testEPR([0,0]))
println(testEPR([0,1]))
println(testEPR([1,0]))
println(testEPR([1,1]))

0.7071067811865475
0.0
0.0
0.7071067811865475


In [ ]:
a = [0.0  0.0  0.707107  -0.707107]
a = reshape(a, 1, 2, 2, 1)
u, v = tsvd(a, (3,2))

##### PROBLEM
1. contract!返回的参数eltype不知道为什么会变成Any，因此在执行tsvd之前必须要用Number进行转换，如tsvd中第一步所示